<a href="https://colab.research.google.com/github/Melvinchen0404/Chess_engine/blob/main/negamax_with_static_evaluation_and_quiescence_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install python-chess

import chess
import random
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Global piece values
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

# Zobrist Hashing Setup
PIECE_TYPES = 6  # pawn, knight, bishop, rook, queen, king
BOARD_SIZE = 64
ZOBRIST_TABLE = np.random.randint(2**63, size=(2, PIECE_TYPES, BOARD_SIZE), dtype=np.uint64)

# Function to calculate the initial Zobrist key
def initial_zobrist_key(board):
    key = np.uint64(0)  # Ensure the key is explicitly of type uint64
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_type = piece.piece_type - 1  # 0-based index
            color = 0 if piece.color == chess.WHITE else 1
            key ^= np.uint64(ZOBRIST_TABLE[color, piece_type, square])
    return key

# Zobrist hash update
def update_zobrist(zobrist_key, move, board):
    print(f"Before update: {zobrist_key}")
    zobrist_key = np.uint64(zobrist_key)  # Ensure key is of type uint64

    # Update based on the piece moved
    piece = board.piece_at(move.from_square)
    if piece:
        piece_type = piece.piece_type - 1  # 0-based index
        zobrist_key ^= np.uint64(ZOBRIST_TABLE[0 if piece.color == chess.WHITE else 1, piece_type, move.from_square])
        zobrist_key ^= np.uint64(ZOBRIST_TABLE[0 if piece.color == chess.WHITE else 1, piece_type, move.to_square])

    # Update based on the piece captured, if any
    captured_piece = board.piece_at(move.to_square)
    if captured_piece:
        piece_type = captured_piece.piece_type - 1
        zobrist_key ^= np.uint64(ZOBRIST_TABLE[0 if captured_piece.color == chess.WHITE else 1, piece_type, move.to_square])

    print(f"After update: {zobrist_key}")
    return zobrist_key

# Transposition Table
TRANSPOSITION_TABLE = {}

KILLER_MOVES = [{} for _ in range(100)]  # Assuming max depth of 100

def store_in_transposition_table(zobrist_key, eval, depth, tt_type):
    # Store the evaluation in the Transposition Table with the evaluation type (exact, upper, or lower bound)
    TRANSPOSITION_TABLE[zobrist_key] = {'eval': eval, 'depth': depth, 'type': tt_type}

def update_killer_moves(depth, move):
    if move not in KILLER_MOVES[depth]:
        if len(KILLER_MOVES[depth]) >= 2:
            KILLER_MOVES[depth].pop(next(iter(KILLER_MOVES[depth])))  # Remove the oldest killer move
        KILLER_MOVES[depth][move] = 0

# Move ordering
def sort_moves(board, depth):
    def move_score(move):
        # Higher score for killer moves
        if move in KILLER_MOVES[depth]:
            return 1000  # Arbitrary high value for killer moves

        # Captures (MVV-LVA heuristic)
        if board.is_capture(move):
            victim_piece = board.piece_at(move.to_square)
            if victim_piece:  # Check if there's a piece at the target square
                victim = victim_piece.piece_type
                attacker = board.piece_at(move.from_square).piece_type
                return 10 * victim - attacker
            else:
                return 0  # If no piece was captured (shouldn't happen for a capture)

        # Promotions
        if move.promotion:
            return 900

        # Checks
        if board.gives_check(move):
            return 500

        # Penalize moves leaving pieces hanging
        board.push(move)
        if not board.is_attacked_by(not board.turn, move.to_square):
            board.pop()
            return -1000  # Arbitrary low value for bad moves
        board.pop()

        return 0  # Default score for other moves

    return sorted(board.legal_moves, key=move_score, reverse=True)

# Negamax with Transposition Table and TT Types
def negamax(board, depth, alpha, beta, zobrist_key):
    color = 1 if board.turn else -1

    # Base case: leaf node or game over
    if depth == 0 or board.is_game_over():
        return quiescence_search(board, alpha, beta, color), None

    # Check for TT hit
    if zobrist_key in TRANSPOSITION_TABLE:
        tt_entry = TRANSPOSITION_TABLE[zobrist_key]
        if tt_entry['depth'] >= depth:
            if tt_entry['type'] == 'exact':
                print(f"TT hit (exact) for zobrist_key {zobrist_key}: {tt_entry['eval']}")
                return tt_entry['eval'], None
            elif tt_entry['type'] == 'lower' and tt_entry['eval'] > alpha:
                print(f"TT hit (lower bound) for zobrist_key {zobrist_key}: {tt_entry['eval']}")
                alpha = max(alpha, tt_entry['eval'])
            elif tt_entry['type'] == 'upper' and tt_entry['eval'] < beta:
                print(f"TT hit (upper bound) for zobrist_key {zobrist_key}: {tt_entry['eval']}")
                beta = min(beta, tt_entry['eval'])

    max_eval = -float('inf')
    best_move = None

    # Sort moves with killer heuristic integration
    for move in sort_moves(board, depth):
        board.push(move)
        zobrist_key = update_zobrist(zobrist_key, move, board)

        eval = -negamax(board, depth - 1, -beta, -alpha, zobrist_key)[0]
        board.pop()

        # Store the evaluation in the transposition table with exact bound
        store_in_transposition_table(zobrist_key, eval, depth, 'exact')

        if eval > max_eval:
            max_eval = eval
            best_move = move

        alpha = max(alpha, eval)
        if alpha >= beta:
            update_killer_moves(depth, move)  # Update killer moves after beta cutoff
            break  # Beta cutoff

    return max_eval, best_move

# Quiescence search
def quiescence_search(board, alpha, beta, color):
    stand_pat = color * static_evaluation(board)
    if stand_pat >= beta:
        return beta
    if alpha < stand_pat:
        alpha = stand_pat

    for move in board.legal_moves:
        if not board.is_capture(move) and not move.promotion and not board.gives_check(move):
            continue
        board.push(move)
        score = -quiescence_search(board, -beta, -alpha, -color)
        board.pop()

        if score >= beta:
            return beta
        if score > alpha:
            alpha = score

    return alpha

# Static evaluation heuristic
def static_evaluation(board):
    material_score = 0

    for square, piece in board.piece_map().items():
        value = piece_values[piece.piece_type]
        material_score += value if piece.color == chess.WHITE else -value

    # Penalize unprotected pieces
    for square, piece in board.piece_map().items():
        if not board.is_attacked_by(not piece.color, square):
            continue
        if not board.is_attacked_by(piece.color, square):  # Piece is hanging
            penalty = piece_values[piece.piece_type] // 2  # Half the piece value as penalty
            material_score -= penalty if piece.color == chess.WHITE else -penalty

    return material_score

# Parallelize root move evaluations
# Parallelize root move evaluations with Transposition Table check
def parallel_search(board, depth):
    zobrist_key = initial_zobrist_key(board)  # Ensure zobrist_key is initialized
    futures = []
    with ThreadPoolExecutor() as executor:
        for move in board.legal_moves:
            new_board = board.copy()
            new_board.push(move)
            new_key = update_zobrist(zobrist_key, move, new_board)

            # Debug Transposition Table
            if new_key in TRANSPOSITION_TABLE:
                print(f"TT hit for move {move}: {TRANSPOSITION_TABLE[new_key]}")

            future = executor.submit(negamax, new_board, depth - 1, -float('inf'), float('inf'), new_key)
            futures.append((future, move))

        best_eval = -float('inf')
        best_move = None
        for future, move in futures:
            eval, _ = future.result()
            if eval > best_eval:
                best_eval = eval
                best_move = move
    return best_move

# Main function to get the best move with iterative deepening
def get_best_move(board, max_depth):
    zobrist_key = initial_zobrist_key(board)
    best_move = None

    for depth in range(1, max_depth + 1):
        best_move = parallel_search(board, depth)
        print(f"Depth {depth} completed.")

        # Debug Transposition Table at root level
        if zobrist_key in TRANSPOSITION_TABLE:
            print(f"TT hit at root: {TRANSPOSITION_TABLE[zobrist_key]}")

    return best_move

# Example usage with a chess library like python-chess
# Define a position using FEN
fen = "rnbqkbnr/ppppp2p/5pp1/4Q3/4P3/8/PPPP1PPP/RNB1KBNR w KQkq - 0 1"
board = chess.Board(fen)

best_move = get_best_move(board, max_depth=5)
print("Best move:", best_move)

Before update: 6740284140324073037
After update: 140487593141132311
Before update: 6740284140324073037
After update: 7300347994034050641
Before update: 6740284140324073037
After update: 5123147715882541290
Before update: 6740284140324073037
After update: 8449003806419747483
Before update: 6740284140324073037
After update: 489570228949925921
Before update: 6740284140324073037
After update: 7209169002310847006
Before update: 6740284140324073037
After update: 1577533526366639927
Before update: 6740284140324073037
After update: 6542436507938042863
Before update: 6740284140324073037
After update: 6012954498386994205
Before update: 6740284140324073037
After update: 957761492521618728
Before update: 6740284140324073037
After update: 5464590255186337410
Before update: 6740284140324073037
After update: 2263243561388414304
Before update: 6740284140324073037
After update: 6347945824551129856
Before update: 6740284140324073037
After update: 7086165978444610532
Before update: 6740284140324073037
Af

NameError: name 'piece_values' is not defined

In [78]:
!pip install python-chess

import chess
import random
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Zobrist Hashing Setup
PIECE_TYPES = 6  # pawn, knight, bishop, rook, queen, king
BOARD_SIZE = 64
ZOBRIST_TABLE = np.random.randint(2**63, size=(2, PIECE_TYPES, BOARD_SIZE), dtype=np.uint64)

# Function to calculate the initial Zobrist key
def initial_zobrist_key(board):
    key = np.uint64(0)  # Ensure the key is explicitly of type uint64
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_type = piece.piece_type - 1  # 0-based index
            color = 0 if piece.color == chess.WHITE else 1
            key ^= np.uint64(ZOBRIST_TABLE[color, piece_type, square])
    return key

# Zobrist hash update
def update_zobrist(zobrist_key, move, board):
    zobrist_key = np.uint64(zobrist_key)  # Ensure key is of type uint64

    # Update based on the piece moved
    piece = board.piece_at(move.from_square)
    if piece:
        piece_type = piece.piece_type - 1  # 0-based index
        zobrist_key ^= np.uint64(ZOBRIST_TABLE[0 if piece.color == chess.WHITE else 1, piece_type, move.from_square])
        zobrist_key ^= np.uint64(ZOBRIST_TABLE[0 if piece.color == chess.WHITE else 1, piece_type, move.to_square])

    # Update based on the piece captured, if any
    captured_piece = board.piece_at(move.to_square)
    if captured_piece:
        piece_type = captured_piece.piece_type - 1
        zobrist_key ^= np.uint64(ZOBRIST_TABLE[0 if captured_piece.color == chess.WHITE else 1, piece_type, move.to_square])

    return zobrist_key

# Transposition Table
TRANSPOSITION_TABLE = {}

KILLER_MOVES = [{} for _ in range(100)]  # Assuming max depth of 100

def store_in_transposition_table(zobrist_key, eval, depth, tt_type):
    # Store the evaluation in the Transposition Table with the evaluation type (exact, upper, or lower bound)
    TRANSPOSITION_TABLE[zobrist_key] = {'eval': eval, 'depth': depth, 'type': tt_type}

def update_killer_moves(depth, move):
    if move not in KILLER_MOVES[depth]:
        if len(KILLER_MOVES[depth]) >= 2:
            KILLER_MOVES[depth].pop(next(iter(KILLER_MOVES[depth])))  # Remove the oldest killer move
        KILLER_MOVES[depth][move] = 0

def evaluate_move(board, move, color):
    """
    Evaluate a move considering material gain/loss, checks, and escaping threats.
    """
    # Initialize score
    score = 0

    # Check if move is a capture
    captured_piece = board.piece_at(move.to_square)
    if captured_piece:
        # Material score: positive for capturing pieces, negative for losing pieces
        score += material_value(captured_piece)

    # Check if the move puts the opponent's king in check
    if board.gives_check(move):
        score += 1000  # High priority for checks

    # Check if the move helps escape threats
    if is_threatened(board, move.from_square, color):
        score -= 500  # Penalize moves that do not escape threats

    # Additional logic for retreating threatened pieces
    if is_threatened(board, move.from_square, color):
        score -= 200  # Penalize moves that leave the piece threatened

    # Add any other positional evaluation logic here, e.g., castling, piece development
    # Placeholder for other evaluations (e.g., king safety)

    return score


def material_value(piece):
    """
    Return the material value of the piece. Positive for opponent's pieces,
    negative for our pieces.
    """
    material = {
      chess.PAWN: 100,
      chess.KNIGHT: 320,
      chess.BISHOP: 330,
      chess.ROOK: 500,
      chess.QUEEN: 900,
      chess.KING: 20000
    }
    return material.get(piece.piece_type, 0)


def is_threatened(board, square, color):
    """
    Returns True if the piece on the square is under threat by the opponent.
    """
    # Check if the square is attacked by any of the opponent's pieces
    opponent_color = chess.WHITE if color == chess.BLACK else chess.BLACK
    for move in board.legal_moves:
        if move.to_square == square and board.piece_at(move.from_square).color == opponent_color:
            return True
    return False

# Move ordering
def sort_moves(board, depth, color):
    def move_score(move):
        # Higher score for killer moves
        if move in KILLER_MOVES[depth]:
            return 1000  # Arbitrary high value for killer moves

        # Captures (MVV-LVA heuristic)
        if board.is_capture(move):
            victim_piece = board.piece_at(move.to_square)
            if victim_piece is not None:  # Check if the target square has a piece
                victim_value = material_value(victim_piece)
                attacker_value = material_value(board.piece_at(move.from_square))
                return victim_value - attacker_value

        # Promotions (a move that promotes a pawn)
        if move.promotion:
            return 900  # Large bonus for promotions

        # Checks (a move that gives check to the opponent's king)
        if board.gives_check(move):
            return 500  # High priority for checks

        # Penalize moves leaving pieces hanging
        board.push(move)
        if not board.is_attacked_by(not board.turn, move.to_square):
            board.pop()
            return -1000  # Large penalty for moves that don't protect pieces
        board.pop()

        # King safety: penalize moves that don't escape threats
        if is_threatened(board, move.from_square, color):
            return -500  # Penalize for not escaping threats

        return 0  # Default score for other moves

    # Sort the moves based on their score: highest score first
    sorted_moves = sorted(board.legal_moves, key=move_score, reverse=True)
    return sorted_moves

# Negamax with Transposition Table and TT Types
def negamax(board, depth, alpha, beta, zobrist_key):
    # Determine the color based on the board's turn
    color = 1 if board.turn else -1  # 1 for white, -1 for black

    # Base case: leaf node or game over
    if depth == 0 or board.is_game_over():
        return quiescence_search(board, alpha, beta, color), None

    # Check for TT hit
    if zobrist_key in TRANSPOSITION_TABLE:
        tt_entry = TRANSPOSITION_TABLE[zobrist_key]
        if tt_entry['depth'] >= depth:
            if tt_entry['type'] == 'exact':
                return tt_entry['eval'], None
            elif tt_entry['type'] == 'lower' and tt_entry['eval'] > alpha:
                alpha = max(alpha, tt_entry['eval'])
            elif tt_entry['type'] == 'upper' and tt_entry['eval'] < beta:
                beta = min(beta, tt_entry['eval'])

    max_eval = -float('inf')
    best_move = None

    # Sort moves with killer heuristic integration, passing color
    for move in sort_moves(board, depth, color):  # Pass color here
        board.push(move)
        zobrist_key = update_zobrist(zobrist_key, move, board)

        eval = -negamax(board, depth - 1, -beta, -alpha, zobrist_key)[0]
        board.pop()

        # Store the evaluation in the transposition table with exact bound
        store_in_transposition_table(zobrist_key, eval, depth, 'exact')

        if eval > max_eval:
            max_eval = eval
            best_move = move

        alpha = max(alpha, eval)
        if alpha >= beta:
            update_killer_moves(depth, move)  # Update killer moves after beta cutoff
            break  # Beta cutoff

    return max_eval, best_move

# Quiescence search
def quiescence_search(board, alpha, beta, color):
    # Get the evaluation components and total score
    material_score, queen_safety_score, king_safety_score, pawn_structure_score, mobility_score, space_control_score, total_score = static_evaluation(board)

    # Use the material_score or any other evaluation component you prefer
    stand_pat = color * material_score  # Or any other score you want to use

    if stand_pat >= beta:
        return beta
    if alpha < stand_pat:
        alpha = stand_pat

    for move in board.legal_moves:
        if not board.is_capture(move) and not move.promotion and not board.gives_check(move):
            continue
        board.push(move)
        score = -quiescence_search(board, -beta, -alpha, -color)
        board.pop()

        if score >= beta:
            return beta
        if score > alpha:
            alpha = score

    return alpha

# Static evaluation heuristic (material, king safety, pawn structure, mobility, space control)
def king_safety(board, color):
    king_square = board.king(color)
    king_x, king_y = divmod(king_square, 8)

    safe_squares = 0
    threatened_squares = 0
    castling_bonus = 0
    rook_movement_penalty = 0

    # Check if the king can castle (either kingside or queenside)
    if board.has_kingside_castling_rights(color):
        castling_bonus += 500  # Bonus for the possibility of kingside castling

        # Check if the rook has been moved or obstructed (if rook has moved, castling is no longer possible)
        if board.piece_at(chess.H1) is None or board.piece_at(chess.H1).color != color:
            rook_movement_penalty += 1000  # Strong penalty if rook has moved or is captured
        if board.piece_at(chess.F1) is not None:
            rook_movement_penalty += 500  # Penalize if the square for kingside castling is blocked

    if board.has_queenside_castling_rights(color):
        castling_bonus += 400  # Bonus for the possibility of queenside castling

        # Check if the rook has been moved or obstructed (if rook has moved, castling is no longer possible)
        if board.piece_at(chess.A1) is None or board.piece_at(chess.A1).color != color:
            rook_movement_penalty += 1000  # Strong penalty if rook has moved or is captured
        if board.piece_at(chess.B1) is not None or board.piece_at(chess.C1) is not None:
            rook_movement_penalty += 500  # Penalize if the squares for queenside castling are blocked

    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if 0 <= king_x + dx < 8 and 0 <= king_y + dy < 8:
                neighbor_square = (king_x + dx) * 8 + (king_y + dy)
                piece = board.piece_at(neighbor_square)
                if piece:
                    if piece.piece_type == chess.PAWN and piece.color == color:
                        safe_squares += 1
                    elif piece.color != color:
                        threatened_squares += 1

    # Adjust the king safety score with the castling bonus and rook movement penalty
    return (2 * safe_squares - 2 * threatened_squares) + castling_bonus - rook_movement_penalty

def pawn_structure(board, color):
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.piece_type == chess.PAWN and piece.color == color:
            x, y = divmod(square, 8)
            isolated = True
            for dx in [-1, 1]:
                for dy in [-1, 0, 1]:
                    neighbor = (x + dx) * 8 + (y + dy)
                    if 0 <= neighbor < 64 and board.piece_at(neighbor) and board.piece_at(neighbor).piece_type == chess.PAWN and board.piece_at(neighbor).color == color:
                        isolated = False
                        break
            if isolated:
                score -= 10
    return score

def mobility(board, color):
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.color == color:
            legal_moves = len(list(board.legal_moves))
            if piece.piece_type == chess.PAWN:
                score += 1 if legal_moves > 1 else 0
            elif piece.piece_type == chess.KNIGHT:
                score += legal_moves
            elif piece.piece_type == chess.QUEEN:
                score += legal_moves * 1.2
    return score

def space_control(board, color):
    control_score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.color == color:
            control_score += len(list(board.attacks(square)))
    return control_score

def static_evaluation(board):
    material_score = 0
    piece_values = {chess.PAWN: 100, chess.KNIGHT: 320, chess.BISHOP: 330, chess.ROOK: 500, chess.QUEEN: 900, chess.KING: 20000}

    for square, piece in board.piece_map().items():
        value = piece_values.get(piece.piece_type, 0)  # Default to 0 if piece type is not found
        material_score += value if piece.color == chess.WHITE else -value

    queen_safety_score = 0
    if queen_under_attack(board, chess.WHITE):
        queen_safety_score -= 500
    if queen_under_attack(board, chess.BLACK):
        queen_safety_score -= 500

    pawn_structure_score = pawn_structure(board, chess.WHITE) if board.turn == chess.WHITE else pawn_structure(board, chess.BLACK)
    king_safety_score = king_safety(board, chess.WHITE) - king_safety(board, chess.BLACK)
    mobility_score = mobility(board, chess.WHITE) if board.turn == chess.WHITE else mobility(board, chess.BLACK)
    space_control_score = space_control(board, chess.WHITE) if board.turn == chess.WHITE else space_control(board, chess.BLACK)

    total_score = (material_score * 1.5) + \
                  (queen_safety_score * 2) + \
                  (king_safety_score * 0.8) + \
                  (pawn_structure_score * 0.6) + \
                  (mobility_score * 0.3) + \
                  (space_control_score * 0.4)

    # Return all components as a tuple
    return material_score, queen_safety_score, king_safety_score, pawn_structure_score, mobility_score, space_control_score, total_score

# Parallelize root move evaluations
def parallel_search(board, depth):
    zobrist_key = initial_zobrist_key(board)
    futures = []
    with ThreadPoolExecutor() as executor:
        for move in board.legal_moves:
            new_board = board.copy()
            new_board.push(move)
            new_key = update_zobrist(zobrist_key, move, new_board)
            future = executor.submit(negamax, new_board, depth - 1, -float('inf'), float('inf'), new_key)
            futures.append((future, move))

        best_eval = -float('inf')
        best_move = None
        for future, move in futures:
            eval, _ = future.result()
            if eval > best_eval:
                best_eval = eval
                best_move = move
    return best_move

# Main function to get the best move with iterative deepening
def get_best_move(board, max_depth):
    zobrist_key = initial_zobrist_key(board)
    best_move = None

    for depth in range(1, max_depth + 1):
        best_move = parallel_search(board, depth)

    return best_move

# Example usage with a chess library like python-chess
fen = "r1b1k3/ppp1pp2/2p2r1Q/8/3q3b/3P4/PPP1PP1P/RN3K2 b KQkq - 0 1"
board = chess.Board(fen)
best_move = get_best_move(board, max_depth=5)

# Correct unpacking of the static evaluation result
material_score, queen_safety_score, king_safety_score, pawn_structure_score, mobility_score, space_control_score, total_score = static_evaluation(board)

print("Best move:", best_move)
print(f"Material score: {material_score}")
print(f"Queen safety score: {queen_safety_score}")
print(f"King safety score: {king_safety_score}")
print(f"Pawn structure score: {pawn_structure_score}")
print(f"Mobility score: {mobility_score}")
print(f"Space control score: {space_control_score}")
print(f"Total evaluation score: {total_score}")

Best move: f6h6
Material score: -740
Queen safety score: -500
King safety score: 1100
Pawn structure score: -30
Mobility score: 58.8
Space control score: 60
Total evaluation score: -1206.36
